## VAE using pytorch

### Data exploration

In [2]:
import pandas as pd
import numpy as np
import os


In [3]:
os.chdir(r"D:\Yichen_Guo\YichenG_Code\Yichen-Capstone-Project\Data")
os.getcwd()

In [9]:
mRNA = pd.read_csv("HCC-GU/mRNA_Enzy.csv",delimiter=";")
miRNA = pd.read_csv("HCC-GU/miRNA_Enzy.csv",delimiter=";")
score_mat =  pd.read_csv("HCC-GU/Score_mat_Enzy.csv",delimiter=";")
patient_labels = pd.read_csv("HCC-GU/sample_labels.csv",delimiter=";") # 1 indicated patient with HCC and CIRR (39), 0 indicated patients with CIRR (25) 

In [10]:
mRNA.head()

## each row is a liver tissue that extract from the 64 patients (marked in the patient_label)

,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AADAT,AANAT,AARS,AASDHPPT,...,ZNF254,ZNF43,ZNF610,ZNF675,ZNF682,ZNF708,ZNF85,ZNF91,ZNF93,ZWILCH
T_001,-0.943365,1.071710,1.071710,0.393936,1.005661,-1.393830,-0.621972,-0.621972,-1.620037,-1.498261,...,1.005661,1.498261,1.393830,0.101446,0.827756,1.393830,0.527832,0.020259,1.218445,-0.670934
T_007,-0.773586,0.670934,0.393936,-1.218445,0.827756,-0.574326,-0.308231,0.183245,1.005661,-1.964241,...,-0.393936,0.721399,0.224533,0.142238,1.393830,0.060807,1.142291,2.269380,0.773586,0.224533
T_013,0.020259,-0.060807,-0.974085,-0.773586,-1.964241,0.943365,0.574326,-0.621972,-0.670934,-1.393830,...,0.308231,-0.101446,0.183245,-1.071710,-0.527832,0.101446,0.884222,0.142238,-0.308231,-0.621972
T_016,-0.621972,-0.350792,-0.183245,0.020259,-0.266171,0.101446,0.670934,1.142291,-1.393830,-0.308231,...,1.142291,-1.071710,-1.301604,0.183245,1.620037,0.266171,-0.574326,1.005661,0.224533,1.620037
T_020,-0.670934,-0.393936,0.266171,0.183245,-0.437756,1.071710,-0.437756,-0.621972,-0.142238,-0.884222,...,1.768555,0.527832,0.482351,0.224533,-0.020259,0.574326,0.393936,1.218445,-0.266171,0.060807


In [14]:
miRNA

## each row is a liver tissue that extract from the 64 patients (marked in the patient_label)

,hsa.let.7a.2.3p,hsa.let.7a.3p,hsa.let.7a.5p,hsa.let.7b.3p,hsa.let.7b.5p,hsa.let.7c.3p,hsa.let.7c.5p,hsa.let.7d.3p,hsa.let.7d.5p,hsa.let.7e.3p,...,hsa.miR.95.3p,hsa.miR.95.5p,hsa.miR.9500,hsa.miR.96.5p,hsa.miR.98.3p,hsa.miR.98.5p,hsa.miR.99a.3p,hsa.miR.99a.5p,hsa.miR.99b.3p,hsa.miR.99b.5p
T_001,1.973054,0.395704,-0.352366,1.400084,0.352366,0.352366,-0.439720,1.223912,0.101901,1.400084,...,-0.576903,-1.223912,-0.020350,0.724636,1.627306,-0.576903,-0.395704,-1.504984,1.147417,0.947598
T_007,-1.973054,0.352366,1.307444,0.020350,0.777057,0.309614,0.439720,0.309614,1.776490,0.831470,...,1.223912,0.309614,-1.042831,1.307444,0.724636,1.776490,0.061080,-0.888189,0.576903,0.439720
T_013,1.010173,0.184067,0.309614,0.439720,0.184067,-0.673945,-0.020350,-0.947598,-0.101901,-0.267365,...,0.061080,-0.061080,1.400084,-0.530201,0.831470,0.624763,-0.439720,-0.439720,-0.225540,-0.530201
T_016,2.279563,-2.279563,-0.225540,-0.309614,-0.395704,-0.576903,0.020350,1.147417,-0.061080,-2.279563,...,-0.624763,-0.439720,-1.042831,-0.831470,-1.223912,-0.831470,-0.888189,-1.147417,-1.504984,-1.504984
T_020,0.673945,-0.184067,-0.267365,-0.888189,-0.576903,-0.225540,-0.184067,-0.624763,-0.624763,-0.309614,...,-0.724636,0.888189,-1.042831,-0.352366,1.307444,0.831470,-0.724636,0.395704,-0.142876,0.225540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T_086,0.530201,1.223912,1.627306,0.352366,1.504984,1.307444,1.504984,-0.530201,0.777057,1.223912,...,1.400084,0.267365,0.777057,-0.061080,-0.020350,0.267365,1.627306,1.776490,1.307444,1.223912
T_089,-1.307444,-0.484515,0.888189,-0.101901,1.400084,-0.020350,0.888189,0.888189,0.309614,-0.061080,...,0.888189,0.673945,-1.042831,0.184067,-0.777057,-0.184067,0.673945,0.777057,0.888189,0.777057
T_092,-1.076518,-0.309614,1.400084,1.147417,1.776490,0.724636,0.267365,0.777057,1.627306,1.973054,...,1.010173,-1.776490,-0.184067,0.309614,-0.530201,-0.624763,0.484515,1.400084,1.973054,1.973054
T_095,-0.777057,0.724636,1.973054,0.484515,1.973054,1.973054,1.627306,1.010173,1.010173,0.439720,...,1.776490,0.624763,-1.042831,0.061080,-0.673945,-0.225540,1.223912,0.831470,1.627306,1.627306


In [13]:
score_mat

## Domain knowledge
## four confiendence level for mRNA-miRNA associations, 1 for expertimentally observed links, 0.75 for highly predicted, 0.5 for moderately predicveted, and 0 for those neirther observed nor predicted

,hsa.let.7a.2.3p,hsa.let.7a.3p,hsa.let.7a.5p,hsa.let.7b.3p,hsa.let.7b.5p,hsa.let.7c.3p,hsa.let.7c.5p,hsa.let.7d.3p,hsa.let.7d.5p,hsa.let.7e.3p,...,hsa.miR.95.3p,hsa.miR.95.5p,hsa.miR.9500,hsa.miR.96.5p,hsa.miR.98.3p,hsa.miR.98.5p,hsa.miR.99a.3p,hsa.miR.99a.5p,hsa.miR.99b.3p,hsa.miR.99b.5p
A2M,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.00,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0
A4GALT,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.00,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0
A4GNT,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.75,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0
AAAS,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.00,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0
AACS,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.00,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF708,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.00,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0
ZNF85,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.00,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0
ZNF91,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.00,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0
ZNF93,0.0,0.0,0.0,0,0,0.0,0,0.0,0,0.00,...,0.0,0.0,0.0,0,0,0,0.0,0,0,0


## building the autoencoder

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, z_dim)
        self.fc_log_var = nn.Linear(hidden_dim, z_dim)
    
    def forward(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc_mu(h)
        log_var = self.fc_log_var(h)
        return mu, log_var

class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h = torch.relu(self.fc1(x))
        recon = torch.sigmoid(self.fc2(h))
        return recon

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, z_dim)
        self.decoder = Decoder(z_dim, hidden_dim, input_dim)
    
    def forward(self, x):
        mu, log_var = self.encoder(x)
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        z = mu + eps * std
        recon = self.decoder(z)
        return recon, mu, log_var

def loss_function(recon_x, x, mu, log_var):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD